![1](images/BOSON_BEACH_400_1.png)
    
![2](images/BOSON_BEACH_400_2.png)

![3](images/BOSON_BEACH_400_3.png)
    
![4](images/BOSON_BEACH_400_4.png)

In [2]:
import json
import pennylane as qml
import pennylane.numpy as np


# Put any helper functions here that you want to make #
def eigen_value(matrix_A):
    evalue,evector = np.linalg.eig(matrix_A)
    return evalue
def encode_hermitian(A, wires):
    """
    Encodes a hermitian matrix A as a unitary U = e^{iA}.

    Args
        - A (numpy.tensor): a 2x2 matrix
        - b (numpy.tensor): a length-2 vector

    Returns
        - (qml.Operation): a unitary operation U = e^{iA}
    """
    return qml.exp(qml.Hermitian(A, wires=wires), coeff=1j)


def mint_to_lime(A, b):
    """
    Calculates the optimal mint and lime proportions in the Mojito HHLime twist.

    Args
        - A (numpy.tensor): a 2x2 matrix
        - b (numpy.tensor): a length-2 vector

    Returns
        - x (numpy.tensor): the solution to Ax = b
        (int): the number of operations in your HHL circuit.
    """
    b_qubits = 1
    b_wires = [0]

    qpe_qubits = 10
    qpe_wires = list(range(b_qubits, b_qubits + qpe_qubits))

    ancilla_qubits = 1
    ancilla_wires = list(
        range(b_qubits + qpe_qubits, ancilla_qubits + b_qubits + qpe_qubits)
    )

    all_wires = b_wires + qpe_wires + ancilla_wires
    dev = qml.device("default.qubit", wires=all_wires)

    @qml.qnode(dev)
    def HHL(A, b):
        """
        Implements the HHL algorithm.
        Args
            - A (numpy.tensor): a 2x2 matrix
            - b (numpy.tensor): a length-2 vector

        Returns
            - (numpy.tensor):
                The probability distribution for the vector x, which is the
                solution to Ax = b.
        """

        # Put your code here #
        # qml.StatePrep(b,wires = b_wires)
        # qml.QuantumPhaseEstimation(encode_hermitian(A, wires=b_wires), estimation_wires=qpe_wires)

        # # U = [[np.cos(theta/2),-np.sin(theta/2)],[np.sin(theta/2),np.cos(theta/2)]]
        # controls = [[qpe_wires[0]],[qpe_wires[1]],[qpe_wires[0],qpe_wires[1]],[qpe_wires[2]],[qpe_wires[0],qpe_wires[2]],[qpe_wires[1],qpe_wires[2]],[qpe_wires[0],qpe_wires[1],qpe_wires[2]],[qpe_wires[3]],[qpe_wires[0],qpe_wires[3]],[qpe_wires[1],qpe_wires[3]]]
        # for i in range(2):
        #     theta = 2*np.arcsin(1/(i+1))
        #     # theta = np.pi/(2*(8-i))
        #     # theta_1 = 2*np.arcsin(1/1022)
        #     # theta_2 = 2*np.arcsin(1/511)
            
        #     U = [[np.cos(theta/2),-np.sin(theta/2)],[np.sin(theta/2),np.cos(theta/2)]]
        #     # U1 = [[np.cos(theta_1/2),-np.sin(theta_1/2)],[np.sin(theta_1/2),np.cos(theta_1/2)]]
        #     # U2 = [[np.cos(theta_2/2),-np.sin(theta_2/2)],[np.sin(theta_2/2),np.cos(theta_2/2)]]
            
        #     qml.ControlledQubitUnitary(U, control_wires = controls[i], wires= ancilla_wires[0])
            # qml.ControlledQubitUnitary(U1, control_wires = qpe_wires[0:10], wires= ancilla_wires[0])
            # qml.ControlledQubitUnitary(U2, control_wires = qpe_wires[1:11], wires= ancilla_wires[0])

            # qml.CRY(2*np.arcsin(1/(i+1)),wires = (qpe_wires[x for x in controls[i]],ancilla_wires[0])

        # qml.MultiRZ(2*np.arcsin(1/(2)),wires = (qpe_wires[0],qpe_wires[1],ancilla_wires[0])).basis("Y")

        # for wire, i in zip(qpe_wires, range(len(qpe_wires))):
            # print(wire, i, 1/2**i, 2*np.arcsin(1/2**i))
            # qml.CRY(np.pi/(2*(10-i)), [wire, ancilla_wires[0]])
            
        # qml.measure(ancilla_wires, postselect=1)
        # qml.adjoint(qml.QuantumPhaseEstimation(encode_hermitian(A, wires=b_wires), estimation_wires=qpe_wires))

                # Put your code here #
        # 1. State preparation
        angle = 2 * np.arccos(b[0])
        qml.RY(angle, b_wires)

        # 2. QPE
        U = encode_hermitian(A, b_wires)
        qml.QuantumPhaseEstimation(U, estimation_wires=qpe_wires)

        # 3. Ancilla bit rotation
        for i in range(1, 2 ** len(qpe_wires)):
            angle = 2 * np.arcsin(1 / i)

            binary_r = np.binary_repr(i, width=len(qpe_wires))
            control_values = list(map(int, binary_r))

            qml.ctrl(qml.RY, qpe_wires, control_values)(angle, ancilla_wires)

        # 4. Postselect on mid-circuit measurement
        qml.measure(ancilla_wires, postselect=1)

        # 5. Inverse QPE (uncompute)
        qml.adjoint(qml.QuantumPhaseEstimation)(U, estimation_wires=qpe_wires)

        return qml.probs(wires=b_wires)

    # we return probs, but we need the state itself (it will be real-valued)
    x = np.sqrt(HHL(A, b))

    return x, len(HHL.tape._ops)

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    A, b = json.loads(test_case_input)
    output, num_ops = mint_to_lime(np.array(A), np.array(b))
    output = output.tolist()
    output.append(num_ops)
    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    output = solution_output[:-1]
    num_ops = solution_output[-1]
    expected_output = json.loads(expected_output)

    assert num_ops > 4, "Your circuit should have a few more operations!"
    assert np.allclose(output, expected_output, rtol=1e-2)
